<a href="https://colab.research.google.com/github/DiegoSamanezDenis/COMP472-Brain-Tumor-MRI-CNN/blob/main/Brain_MRI_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import os

files.upload()

!mkdir -p ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d ahmedhamada0/brain-tumor-detection
!kaggle datasets download -d fernando2rad/brain-tumor-mri-images-44c
!kaggle datasets download -d rm1000/brain-tumor-mri-scans

!unzip -q brain-tumor-detection.zip -d dataset_br35h
!unzip -q brain-tumor-mri-images-44c.zip -d dataset_44c
!unzip -q brain-tumor-mri-scans.zip -d dataset_mri_scans


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/ahmedhamada0/brain-tumor-detection
License(s): copyright-authors
  0% 0.00/84.0M [00:00<?, ?B/s]
100% 84.0M/84.0M [00:00<00:00, 1.74GB/s]
Dataset URL: https://www.kaggle.com/datasets/fernando2rad/brain-tumor-mri-images-44c
License(s): Community Data License Agreement - Sharing - Version 1.0
 95% 179M/188M [00:00<00:00, 572MB/s] 
100% 188M/188M [00:00<00:00, 609MB/s]
Dataset URL: https://www.kaggle.com/datasets/rm1000/brain-tumor-mri-scans
License(s): CC0-1.0
 63% 149M/236M [00:00<00:00, 1.56GB/s]
100% 236M/236M [00:00<00:00, 895MB/s] 


In [2]:
import torch
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

In [3]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [8]:
def get_stratified_splits(dataset, val_split=0.15, test_split=0.15):
  targets = dataset.targets

  train_idx, temp_idx = train_test_split(
    np.arange(len(targets)),
    test_size=(val_split + test_split),
    stratify = targets,
    random_state=42
  )

  temp_targets = [targets[i] for i in temp_idx]

  val_idx, test_idx = train_test_split(
      temp_idx,
      test_size=0.5,
      stratify = temp_targets,
      random_state=42
  )

  return Subset(dataset, train_idx), Subset(dataset, val_idx), Subset(dataset, test_idx)

In [9]:
ds_br35h = datasets.ImageFolder('dataset_br35h', transform=data_transforms)
train_br, val_br, test_br=get_stratified_splits(ds_br35h)

In [10]:
train_loader_br=DataLoader(train_br, batch_size=32, shuffle=True)
val_loader_br=DataLoader(val_br, batch_size=32, shuffle=False)
test_loader_br=DataLoader(test_br, batch_size=32, shuffle=False)

In [11]:
print(f'Total images: {len(ds_br35h)}')
print(f'Train Size: {len(train_br)}, Val Size: {len(val_br)}, Test Size: {len(test_br)}')

Total images: 3861
Train Size: 2702, Val Size: 579, Test Size: 580
